In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%%capture
!git clone https://github.com/dpcomp-org/dpcomp_core.git

In [ ]:
%%capture
!pip install diffprivlib

In [ ]:
# Remove default module
%%capture
%cd dpcomp_core
!rm -r dpcomp_core

In [ ]:
# Get zipped new module from Jane's server
%%capture
!wget https://dev.universalities.com/dpcomp_core.zip

In [ ]:
# Unzip Jane's version of the module
%%capture
!unzip dpcomp_core.zip

In [ ]:
# Add requirements so pip does the version matching right
!echo diffprivlib >> resources/requirements.txt
!echo pandas >> resources/requirements.txt

In [ ]:
# Install pip requirements
%%capture
!pip install -r resources/requirements.txt

In [ ]:
# Install swig
%%capture
!apt-get install swig

In [ ]:
# Run C++ setup for DAWA
%%capture
%cd dpcomp_core/algorithm/dawa/
!bash setup.sh
%cd ../../../

In [ ]:
import os
os.getcwd()

'/content/dpcomp_core'

In [ ]:
%cd dpcomp_core/algorithm/ahp/
!bash setup.sh
%cd ../../../

/content/dpcomp_core/dpcomp_core/algorithm/ahp
rm: cannot remove 'ahp_fast.c': No such file or directory
rm: cannot remove '*.so': No such file or directory
Compiling lib/ahp_fast.pyx because it changed.
[1/1] Cythonizing lib/ahp_fast.pyx
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/dpcomp_core/dpcomp_core/algorithm/ahp/lib/ahp_fast.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
running build_ext
building 'ahp_fast' extension
creating build
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/lib
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.7m -c lib/ahp_fast.c

In [ ]:
from __future__ import division
from __future__ import print_function
from past.utils import old_div
from dpcomp_core.algorithm import dawa
from dpcomp_core.algorithm import AG
from dpcomp_core.algorithm import ahp
from dpcomp_core import dataset
from dpcomp_core import workload
import numpy as np

In [ ]:
def runDAWA(data, bin, epsilon):
  domain = (bin,bin)
  epsilon = epsilon


  seed = 1
  shape_list = [(5,5),(10,10)]
  size = 5000

  w = workload.RandomRange(shape_list=shape_list, 
                         domain_shape=domain, 
                         size=size, 
                         seed=seed)

  a = dawa.dawa2D_engine()
  H, xedges, yedges = np.histogram2d(data['x'], data['y'], bins = bin)
  H = H.T
  x_hat = a.Run(w, H, epsilon, seed)
  return x_hat

In [ ]:
def runAHP(data, bin, epsilon):
  domain = (bin,bin)

  seed = 1
  shape_list = [(5,5),(10,10)]
  size = 5000

  w = workload.RandomRange(shape_list=shape_list, 
                        domain_shape=domain, 
                        size=size, 
                        seed=seed)

  a = ahp.ahpND_engine(ratio = 0.85, eta = 0.35)
  H, xedges, yedges = np.histogram2d(data['x'], data['y'], bins = bin)
  H = H.T
  x_hat = a.Run(w, H, epsilon, seed)
  return x_hat

In [ ]:
def runAGrid(data, bin, epsilon):
  domain = (bin,bin)
  seed = 1
  shape_list = [(5,5),(10,10)]
  size = 5000

  w = workload.RandomRange(shape_list=shape_list, 
                        domain_shape=domain, 
                        size=size, 
                        seed=seed)

  a = AG.AG_engine(c = 10, c2 = 5, alpha = 0.5)
  H, xedges, yedges = np.histogram2d(data['x'], data['y'], bins = bin)
  H = H.T
  x_hat = a.Run(w, H, epsilon, seed)
  return x_hat

In [ ]:
from diffprivlib.tools import histogram2d
def runGeoTruncated(data, bin, epsilon):
  newList = histogram2d(data['x'],data['y'],epsilon,bin, density = False)
  x_hat = newList[0].T
  return x_hat



In [ ]:
from diffprivlib.mechanisms.laplace import Laplace

from sys import maxsize
def histogramdd(sample, epsilon, bins, range=None, weights=None, density=None, accountant=None, **unused_args):

    hist, bin_edges = np.histogramdd(sample, bins=bins, range=range, normed=None, weights=weights, density=None)

    dp_mech = Laplace().set_epsilon(epsilon).set_sensitivity(1)
    dp_hist = np.zeros_like(hist)
    iterator = np.nditer(hist, flags=['multi_index'])

    while not iterator.finished:
        dp_hist[iterator.multi_index] = dp_mech.randomise(int(iterator[0]))
        iterator.iternext()

    dp_hist = dp_hist.astype(float, casting='safe')

    if density:
        # calculate the probability density function
        dims = len(dp_hist.shape)
        dp_hist_sum = dp_hist.sum()
        for i in np.arange(dims):
            shape = np.ones(dims, int)
            shape[i] = dp_hist.shape[i]
            # noinspection PyUnresolvedReferences
            dp_hist = dp_hist / np.diff(bin_edges[i]).reshape(shape)

        if dp_hist_sum > 0:
            dp_hist /= dp_hist_sum

  

    return dp_hist, bin_edges


In [ ]:
def laplaceHistogram(array_x, array_y, epsilon, bins, range=None, weights=None, density=None, accountant=None,
                **unused_args):

    try:
        num_bins = len(bins)
    except TypeError:
        num_bins = 1

    if num_bins not in (1, 2):
        xedges = yedges = np.asarray(bins)
        bins = [xedges, yedges]

    hist, edges = histogramdd([array_x, array_y], epsilon=epsilon, bins=bins, range=range, weights=weights,
                              density=density, accountant=accountant)
    
    return hist, edges[0], edges[1]


In [ ]:
def runLaplace(data, bin, epsilon):
  newList = laplaceHistogram(data['x'],data['y'],epsilon,bin, density = False)
  x_hat = newList[0].T
  return x_hat


In [ ]:
import pandas as pd

In [ ]:
tasks = ['distribution', 'clusters', 'correlation']
epsilons = [.5, .1, .05, .01]
algorithms = ['DAWA', 'AHP', "AGrid", 'Geometric', 'Laplace']
bins = [32,64]


In [ ]:
chartDictionary = {'0': ['distribution', 'correlation'], '1': ['distribution'], 
                   '2': ['distribution', 'clusters', 'correlation'], '3': ['distribution'],
                   '4': ['distribution', 'clusters'], '5': [ 'correlation'],
                   '6': ['distribution', 'correlation'], '7': ['distribution', 'correlation'],
                   '8':['distribution'], '9': ['distribution'],
                   '10': ['distribution', 'clusters'], '11': ['distribution', 'correlation'],
                   '12': ['correlation'], '13': ['distribution', 'correlation'],
                   '14': ['correlation'], '15': ['distribution'],
                   '16': ['distribution', 'clusters'], '17': ['distribution'],
                   '18': ['clusters'], '19': ['distribution']}

In [ ]:
import matplotlib.pyplot as plt


In [ ]:

from subprocess import run
directory = '/finalData'

for filename in os.scandir(directory):
    if filename.is_file():
        
        # df = pd.read_csv(filename.path)
        for key, value in chartDictionary.items():
          if ('/'+key+'.csv') in (filename.path):

            df = pd.read_csv(filename.path)
            
            H, xedges, yedges = np.histogram2d(df['x'], df['y'], bins = 32)
            H = H.T
            plt.imshow(H, interpolation='nearest', origin='lower', cmap = 'gray_r', aspect = 'auto',
              extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])
            ax = plt.gca()
            ax.axes.xaxis.set_ticklabels([])
            ax.axes.yaxis.set_ticklabels([])
            plt.savefig('/finalImages/'+key +'/' + ''+key+ '_' + "Binned_None_None_32.png")
            plt.clf()   

            H, xedges, yedges = np.histogram2d(df['x'], df['y'], bins = 64)
            H = H.T
            plt.imshow(H, interpolation='nearest', origin='lower', cmap = 'gray_r', aspect = 'auto',
              extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])
            ax = plt.gca()
            ax.axes.xaxis.set_ticklabels([])
            ax.axes.yaxis.set_ticklabels([])
            plt.savefig('/finalImages/'+key +'/' + ''+key+ '_' + "Binned_None_None_64.png")
            plt.clf()

            for v in value:
              for e in epsilons:
                for b in bins:
                  H, xedges, yedges = np.histogram2d(df['x'], df['y'], bins = b)
                  for a in algorithms:
                    if a == 'DAWA':
                      dawaPrivate = runDAWA(df, b,e)
                 
                      plt.imshow(dawaPrivate, interpolation='nearest', origin='lower', cmap = 'gray_r', aspect = 'auto',
                        extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])
                      ax = plt.gca()
                      ax.axes.xaxis.set_ticklabels([])
                      ax.axes.yaxis.set_ticklabels([])
                      plt.savefig('/finalImages/'+key +'/' + key + '_' + v + '_' + a + '_' + str(e) + '_' + str(b) + '.png')
                      plt.clf()


                    if a == 'AHP':
                      ahpPrivate = runAHP(df,b,e)
                   
                      plt.imshow(ahpPrivate, interpolation='nearest', origin='lower', cmap = 'gray_r', aspect = 'auto',
                        extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])
                      ax = plt.gca()
                      ax.axes.xaxis.set_ticklabels([])
                      ax.axes.yaxis.set_ticklabels([])
                      plt.savefig('/finalImages/'+key +'/' + key + '_' + v + '_' + a + '_' + str(e) + '_' + str(b) + '.png')
                      plt.clf()


                    if a == "AGrid":
                      aGridPrivate = runAGrid(df,b,e)
                  
                      plt.imshow(aGridPrivate, interpolation='nearest', origin='lower', cmap = 'gray_r', aspect = 'auto',
                        extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])
                      ax = plt.gca()
                      ax.axes.xaxis.set_ticklabels([])
                      ax.axes.yaxis.set_ticklabels([])
                      plt.savefig('/finalImages/'+key +'/' + key + '_' + v + '_' + a + '_' + str(e) + '_' + str(b) + '.png')
                      plt.clf()

                    if a == "Geometric":
                      geoPrivate = runGeoTruncated(df,b,e)
                   
                      plt.imshow(geoPrivate, interpolation='nearest', origin='lower', cmap = 'gray_r', aspect = 'auto',
                        extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])
                      ax = plt.gca()
                      ax.axes.xaxis.set_ticklabels([])
                      ax.axes.yaxis.set_ticklabels([])
                      plt.savefig('/finalImages/'+key +'/' + key + '_' + v + '_' + a + '_' + str(e) + '_' + str(b) + '.png')
                      plt.clf()

                    if a == 'Laplace':
                      laplacePrivate = runLaplace(df,b,e)
                    
                      plt.imshow(laplacePrivate, interpolation='nearest', origin='lower', cmap = 'gray_r', aspect = 'auto',
                        extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])
                      ax = plt.gca()
                      ax.axes.xaxis.set_ticklabels([])
                      ax.axes.yaxis.set_ticklabels([])
                      plt.savefig('/finalImages/'+key +'/' + key + '_' + v + '_' + a + '_' + str(e) + '_' + str(b) + '.png')
                      plt.clf()

                    
              


/content/dpcomp_core/dpcomp_core/query_nd_union.py:95: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  array[ndRangeUnion.slice(r)] += r.wgt
/usr/local/lib/python3.7/dist-packages/diffprivlib/tools/histograms.py:229: PrivacyLeakWarning: Range parameter has not been specified (or has missing elements). Falling back to taking range from the data.
 To ensure differential privacy, and no additional privacy leakage, the range must be specified for each dimension independently of the data (i.e., using domain knowledge).
  PrivacyLeakWarning)
/usr/local/lib/python3.7/dist-packages/diffprivlib/tools/histograms.py:229: PrivacyLeakWarning: Range parameter has not been specified (or has missing elements). Falling back to taking range from the data.
 To ensure differential pr

<Figure size 432x288 with 0 Axes>